In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [3]:
FS_Score = pd.read_csv("C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/CLEF_2024/Original_data/train_edss.csv")
Static = pd.read_csv('C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/CLEF_2024/Original_data/train_static.csv')
environment = pd.read_csv("C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/CLEF_2024/Original_data/train_environmental_meas.csv")                       

In [4]:
Static.isnull().sum()

patient_id                    0
sex                           0
residence_classification      0
ethnicity                    14
ms_in_pediatric_age           0
age_at_onset                  0
age_at_baseline               0
diagnostic_delay              1
diagnosis_criteria_detail    24
spinal_cord_symptom           0
brainstem_symptom             0
eye_symptom                   0
supratentorial_symptom        0
other_symptoms                0
centre                        0
diagnosis_criteria           24
dtype: int64

In [3]:
FS_static = pd.merge(FS_Score, Static, left_on='patient_id', right_on = "patient_id", how='inner')
FS_static = FS_static[['patient_id','week_from_baseline','edss_as_evaluated_by_clinician','sex','ms_in_pediatric_age', 
                       'age_at_onset','age_at_baseline', 'spinal_cord_symptom', 'brainstem_symptom','eye_symptom',
                       'supratentorial_symptom', 'other_symptoms','pyramidal', 'cerebellar', 'brainstem', 
                       'sensory','bowel_and_bladder', 'visual_function', 'cerebral_functions','ambulation']]# 834 records for 199 patients

# Label encoding on string features
label_encoder = LabelEncoder()
FS_static['sex']= label_encoder.fit_transform(FS_static['sex'])
FS_static['ms_in_pediatric_age']= label_encoder.fit_transform(FS_static['ms_in_pediatric_age'])
FS_static['spinal_cord_symptom']= label_encoder.fit_transform(FS_static['spinal_cord_symptom'])
FS_static['brainstem_symptom']= label_encoder.fit_transform(FS_static['brainstem_symptom'])
FS_static['eye_symptom']= label_encoder.fit_transform(FS_static['eye_symptom'])
FS_static['supratentorial_symptom']= label_encoder.fit_transform(FS_static['supratentorial_symptom'])
FS_static['other_symptoms']= label_encoder.fit_transform(FS_static['other_symptoms'])

In [4]:
imputer = KNNImputer()
imputed_FS_Static = imputer.fit_transform(FS_static.iloc[:,1:])
imputed_FS_Static = pd.DataFrame(imputed_FS_Static, columns = FS_static.columns[1:])

int_columns = [0,2,3,4,5,6,7,8,9,10]
int_features = imputed_FS_Static.iloc[:,int_columns]
data_int_features = int_features.apply(lambda col: col.astype(int) if col.dtype == float else col)
float_features = [col_idx for col_idx, col in enumerate(imputed_FS_Static.columns) if col_idx not in int_columns]
data_float_features = imputed_FS_Static.iloc[:,float_features]
imputed_FS_Static = pd.concat([data_int_features, data_float_features], axis=1)

selected_columns = ["edss_as_evaluated_by_clinician"]
selected_columns.extend(imputed_FS_Static.columns[:10])
selected_columns.extend(imputed_FS_Static.columns[11:])

imputed_FS_Static = imputed_FS_Static[selected_columns]
imputed_FS_Static[imputed_FS_Static.columns[11:]] = imputed_FS_Static[imputed_FS_Static.columns[11:]].round()
imputed_FS_Static = pd.concat([FS_static[["patient_id"]], imputed_FS_Static], axis=1)
imputed_FS_Static.isnull().sum()

,patient_id,edss_as_evaluated_by_clinician,week_from_baseline,sex,ms_in_pediatric_age,age_at_onset,age_at_baseline,spinal_cord_symptom,brainstem_symptom,eye_symptom,supratentorial_symptom,other_symptoms,pyramidal,cerebellar,brainstem,sensory,bowel_and_bladder,visual_function,cerebral_functions,ambulation
0,67396654612589370083623092407810766693,2.0,0,1,1,17,23,0,1,0,1,0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0
1,67396654612589370083623092407810766693,2.0,29,1,1,17,23,0,1,0,1,0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0
2,67396654612589370083623092407810766693,2.0,58,1,1,17,23,0,1,0,1,0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
3,67396654612589370083623092407810766693,2.0,84,1,1,17,23,0,1,0,1,0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0
4,67396654612589370083623092407810766693,2.5,106,1,1,17,23,0,1,0,1,0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0


In [57]:
imputed_FS_Static.to_csv("Imputed_FS_Static_data.csv", index= False)

In [65]:
# Make air pollution who percentage features
Env_features = {'PM25': 1, 'PM10': 2, 'CO': 3, 'NO2': 4, 'O3': 5, 'SO2': 6}

for key in Env_features:
    three_env_feature = environment.filter(like=key)
    environment[key+"_who_percentage"] = three_env_feature.iloc[:,2] / three_env_feature.iloc[:,0]
selected_env_feature = environment.filter(regex='mean|percentage')
environment = pd.concat([environment[["patient_id","week_from_baseline"]], selected_env_feature], axis=1)

# Select data after -4 weeks of baseline
environment = environment[environment["week_from_baseline"] >= -4] # 98456 * 22
environment.shape

(98456, 22)

In [67]:
#KNN imputation K=5 for environment features
imputer = KNNImputer()
KNN_imputed = imputer.fit_transform(environment.iloc[:,2:])
P_id_W_baseline = environment[["patient_id","week_from_baseline"]]
concatenated_array = np.concatenate((P_id_W_baseline, KNN_imputed), axis=1)
result = pd.DataFrame(concatenated_array, columns = environment.columns)
result.head() # 98456 * 22

,patient_id,week_from_baseline,PM25_mean,PM10_mean,CO_mean,NO2_mean,O3_mean,SO2_mean,wind_speed_mean,humidity_mean,...,precipitation_mean,average_temperature_mean,min_temperature_mean,max_temperature_mean,PM25_who_percentage,PM10_who_percentage,CO_who_percentage,NO2_who_percentage,O3_who_percentage,SO2_who_percentage
0,100619256189067386770484450960632124211,-4,27.0,36.777601,1.115048,38.547619,3.454348,3.163423,1.59,64.1,...,0.871429,5.021429,1.497143,8.722857,1.0,0.08,0.0,1.0,0.0,0.0
1,100619256189067386770484450960632124211,-3,26.81125,37.603274,1.010177,34.036491,5.338038,0.768089,1.567143,73.414286,...,2.6,3.577143,1.512857,6.351428,1.0,0.285714,0.0,0.857143,0.0,0.0
2,100619256189067386770484450960632124211,-2,16.295583,24.987321,1.010177,34.077381,17.483757,0.768089,1.528571,74.842859,...,1.442857,6.345714,3.582857,9.935714,0.466667,0.057143,0.0,0.857143,0.0,0.0
3,100619256189067386770484450960632124211,-1,14.819444,31.681548,0.6073,36.641046,42.94396,1.579147,1.525714,65.785714,...,3.657143,9.152857,4.715714,14.101428,0.5,0.142857,0.0,1.0,0.0,0.0
4,100619256189067386770484450960632124211,0,33.128472,29.72063,0.537045,31.47619,35.107143,4.098735,1.661429,72.485715,...,12.157143,11.465714,7.721428,15.897143,0.714286,0.142857,0.0,0.714286,0.0,0.0


In [68]:
result.to_csv("Imputed_environment_data.csv", index= False)